In [ ]:
import pandas as pd
import numpy as np

def calculate_optimal_pnl(df, heat_rate=10, mw=250, min_run_hours=3, min_down_hours=6):
    # Calculate potential PnL for each hour
    df['potential_pnl'] = (df['power_price'] - (df['gas_price'] * heat_rate)) * mw
    
    n = len(df)
    # Initialize columns for decisions and PnL
    df['run_decision'] = False
    df['pnl'] = 0.0
    
    # Dynamic programming to find the optimal operation schedule
    # Initialize a DP array where dp[i] represents the max PnL achievable from hour i to the end
    dp = [0] * (n + 1)
    # To track the operation decisions
    run_hours = [0] * n  # Number of hours to run starting from each hour
    
    for i in range(n - 1, -1, -1):
        max_pnl = 0
        # Calculate not running scenario
        max_pnl = dp[i + 1]
        run_hours[i] = 0
        # Calculate running scenarios, starting with the minimum running hours
        pnl_running = 0
        for j in range(i, min(i + min_run_hours, n)):
            pnl_running += df.iloc[j]['potential_pnl']
        for j in range(i + min_run_hours, n):
            if j < n:
                pnl_running += df.iloc[j]['potential_pnl']
            # Check if continuing to run is profitable
            if j + 1 < n:
                if pnl_running + dp[j + 1] > max_pnl:
                    max_pnl = pnl_running + dp[j + 1]
                    run_hours[i] = j - i + 1
            else:
                if pnl_running > max_pnl:
                    max_pnl = pnl_running
                    run_hours[i] = j - i + 1
        dp[i] = max_pnl
    
    # Apply the decisions
    i = 0
    while i < n:
        if run_hours[i] > 0:
            for j in range(i, i + run_hours[i]):
                df.at[df.index[j], 'pnl'] = df.iloc[j]['potential_pnl']
                df.at[df.index[j], 'run_decision'] = True
            i += run_hours[i]  # Skip the hours the plant has been decided to run
        else:
            df.at[df.index[i], 'pnl'] = 0
            i += 1
    
    return df

# Example DataFrame for demonstration
data = {
    'time': pd.date_range(start='2022-01-01', periods=24, freq='H'),
    'power_price': np.random.uniform(50, 150, 24),  # Simulated power prices
    'gas_price': np.random.uniform(3, 10, 24),      # Simulated gas prices
}
df = pd.DataFrame(data)
df.set_index('time', inplace=True)

# Call the function
optimized_df = calculate_optimal_pnl(df)

# Display the optimized DataFrame
print(optimized_df[['power_price', 'gas_price', 'potential_pnl', 'run_decision', 'pnl']])
